# Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Library Definition

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import VotingClassifier

from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

# Data Upload

In [ ]:
# preprocess type
PCA = 1
FEATURE_REDUC = 2
UNDER_SAMPLE = 3

preprocess_type = PCA

In [ ]:

if preprocess_type == PCA:
  X_model_file_path = "/content/drive/Shareddrives/Intro-data-science/data/X_model_preprocess_pca.csv"
  Y_model_file_path = "/content/drive/Shareddrives/Intro-data-science/data/Y_model.csv"
  X_exam_file_path = "/content/drive/Shareddrives/Intro-data-science/data/X_test_preprocess_pca.csv"

elif preprocess_type == FEATURE_REDUC:
  X_model_file_path = "/content/drive/Shareddrives/Intro-data-science/data/X_model_preprocess_feat_reduc.csv"
  Y_model_file_path = "/content/drive/Shareddrives/Intro-data-science/data/Y_model.csv"
  X_exam_file_path = "/content/drive/Shareddrives/Intro-data-science/data/X_test_preprocess_feat_reduc.csv"

elif UNDER_SAMPLE:
  X_model_file_path = "/content/drive/Shareddrives/Intro-data-science/data/X_model_preprocess_undersampled.csv"
  Y_model_file_path = "/content/drive/Shareddrives/Intro-data-science/data/Y_model_undersampled.csv"
  X_exam_file_path = "/content/drive/Shareddrives/Intro-data-science/data/X_test_preprocess.csv"


In [ ]:
X_model_all = pd.read_csv(X_model_file_path, index_col = 0)

In [ ]:
Y_model_all = pd.read_csv(Y_model_file_path)

# Model 훈련

In [ ]:
rf = RandomForestClassifier(random_state = 100)
xgb = XGBClassifier(random_state = 100)
lgb = LGBMClassifier(n_estimators = 100)

models = [
    ('rf', rf),
    ('XGB', xgb),
    ('LightGBM', lgb)
]

model = VotingClassifier(estimators = models, voting = 'soft')

In [ ]:
cv_valid_score = cross_val_score(model, X_model_all, Y_model_all, scoring = 'roc_auc', cv = 5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (cv_valid_score.mean(), cv_valid_score.std()))

/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressio

0.88 accuracy with a standard deviation of 0.00


# Probability Predication

In [ ]:
X_exam = pd.read_csv(X_exam_file_path, index_col = 0)

In [ ]:
Y_exam_prob_numpy = model.predict_proba(X_exam)
Y_exam_prob = pd.DataFrame(Y_exam_prob_numpy[:, 1])

Y_exam_prob.columns = ['business_prob']

In [ ]:
Y_exam_prob.head()

,business_prob
0,0.042702
1,0.001049
2,0.046102
3,0.002897
4,0.000312


In [ ]:
Y_exam_prob[Y_exam_prob["business_prob"] >= 0.5]

,business_prob
200,0.876321
310,0.810219
323,0.752010
446,0.510747
506,0.543856
...,...
199810,0.724228
199901,0.532878
199928,0.800192
199990,0.613100


In [ ]:
Y_exam_prob.to_csv("/content/drive/Shareddrives/Intro-data-science/data/Y_exam_prob.csv")